In [ ]:
from IPython.display import display, HTML

import pandas as pd
import matplotlib.pyplot as plt

from src.evb import get_evb_incidence, get_evb_sick_notes
from src.rki import get_consultations, get_grippeweb

In [ ]:
sick_notes = get_evb_sick_notes()
# compound data
merged = pd.merge(
    pd.merge(
        get_evb_incidence(),
        get_consultations(),
        on='Week', how='outer'),
    get_grippeweb(),
    on='Week', how='outer'
).sort_values('Week').set_index('Week')
merged.head()

In [ ]:
def _plot_since(date: str):
    ax = merged.loc[pd.to_datetime(date):].plot(
        alpha=0.6,
        figsize=(10, 5),
        title=f'Incidence comparison since {date}',
        xlabel='Week',
        ylabel='7-day incidence'
    )
    ax.legend(loc='upper left')

    from pandas.plotting._matplotlib.style import get_standard_colors
    ax2 = ax.twinx()
    sick_notes.loc[pd.to_datetime(date):].plot(
        ax=ax2,
        color=get_standard_colors(num_colors=len(merged.columns) + 1)[-1],
        alpha=0.6
    )
    
_plot_since('2014')
_plot_since('2020')

In [ ]:
def plot_winter(season: int):
    df =  merged.loc[pd.to_datetime(f'20{season}-10-01'):pd.to_datetime(f'20{season + 1}-03-31')]
    ax = df.pct_change().plot.bar(
        alpha=0.7,
        figsize=(10, 3),
        title=f'Comparison of relative change in incidence, winter 20{season}/{season + 1}',
        rot=0,
        ylabel='Relative change of 7-day incidence'
    )
    tick_labels = ['' if d.day > 7 else d.strftime('%b') for d in df.index]
    ax.set_xticklabels(tick_labels)
    ax.legend(loc='upper right')
    
    ax = df.plot(
        alpha=0.7,
        figsize=(10, 3),
        title=f'Comparison of incidence, winter 20{season}/{season + 1}',
        ylabel='7-day incidence'
    )
    ax.legend(loc='upper right')
    
    df = (df - df.min()) / (df.max() - df.min())
    ax = df.plot(
        alpha=0.7,
        figsize=(10, 3),
        # title=f'Min-Max-normalized 7-day incidence, winter 20{season}/{season + 1}',
        ylabel='Normalized incidence'
    )
    ax.legend(loc='upper right')

In [ ]:
plot_winter(21)